In [16]:
import lightgbm as lgb
import optuna
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
lgb.callback.reset_parameter(verbosity=-1)
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='lightgbm')

In [17]:
df = pd.read_csv("C:/Users/Administrator/Desktop/FinetuningOptuna/heart.csv")
df = pd.get_dummies(df)
X = df.drop("HeartDisease", axis=1).copy()
y = df["HeartDisease"].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=369)


In [18]:
model = lgb.LGBMClassifier()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

[LightGBM] [Info] Number of positive: 358, number of negative: 284
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000135 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 362
[LightGBM] [Info] Number of data points in the train set: 642, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.557632 -> initscore=0.231559
[LightGBM] [Info] Start training from score 0.231559
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

0.8840579710144928

In [19]:
def objective(trial):
    hiper = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 7),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-5, 1e3, log=True),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 1.0),
        'num_leaves': trial.suggest_int('num_leaves', 10, 300),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 6, 50), 
        'verbosity': -1,  
        'device_type': 'gpu',  
        'gpu_platform_id': 0,
        'gpu_device_id': 0
    }
    model = lgb.LGBMClassifier(**hiper)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return acc
stduy = optuna.create_study(direction="maximize")
stduy.optimize(objective, n_jobs=-1, n_trials=1000)

[I 2025-08-22 22:04:37,233] A new study created in memory with name: no-name-57d44b28-2fc2-41f4-add9-27a60c7583f5
[I 2025-08-22 22:04:58,969] Trial 5 finished with value: 0.8876811594202898 and parameters: {'n_estimators': 545, 'max_depth': 11, 'learning_rate': 0.20847289779220926, 'subsample': 0.9114064811896807, 'subsample_freq': 3, 'colsample_bytree': 0.9226951610397237, 'reg_alpha': 1.8280169751937374, 'reg_lambda': 1.8866951031838457e-08, 'min_child_samples': 12, 'min_child_weight': 13.187230201513085, 'min_split_gain': 0.5837696217273665, 'num_leaves': 294, 'feature_fraction': 0.6431801359563468, 'bagging_fraction': 0.8884186003327801, 'bagging_freq': 1, 'min_data_in_leaf': 7}. Best is trial 5 with value: 0.8876811594202898.
[I 2025-08-22 22:04:59,177] Trial 1 finished with value: 0.8804347826086957 and parameters: {'n_estimators': 431, 'max_depth': 15, 'learning_rate': 0.2865064560272265, 'subsample': 0.7043207792228944, 'subsample_freq': 4, 'colsample_bytree': 0.813095283273148

In [20]:
model = lgb.LGBMClassifier(**stduy.best_params)
print(stduy.best_params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc  = accuracy_score(y_test, y_pred)
print(acc)

{'n_estimators': 814, 'max_depth': 11, 'learning_rate': 0.06989560060634954, 'subsample': 0.9482692266623619, 'subsample_freq': 7, 'colsample_bytree': 0.6222682049072605, 'reg_alpha': 5.383072973241785, 'reg_lambda': 0.006398114717178947, 'min_child_samples': 29, 'min_child_weight': 1.2571060035323882, 'min_split_gain': 0.5071743099157269, 'num_leaves': 124, 'feature_fraction': 0.7443293995885548, 'bagging_fraction': 0.8381120278532139, 'bagging_freq': 5, 'min_data_in_leaf': 27}
[LightGBM] [Warning] min_data_in_leaf is set=27, min_child_samples=29 will be ignored. Current value: min_data_in_leaf=27
[LightGBM] [Warning] feature_fraction is set=0.7443293995885548, colsample_bytree=0.6222682049072605 will be ignored. Current value: feature_fraction=0.7443293995885548
[LightGBM] [Warning] bagging_fraction is set=0.8381120278532139, subsample=0.9482692266623619 will be ignored. Current value: bagging_fraction=0.8381120278532139
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=7 wi